<a href="https://colab.research.google.com/github/icculp/route-66-scrape_public/blob/master/historic66.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Should run for 19 pages; if it errors much before try running the script again

In [15]:
# !pip install kora -q
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from kora.selenium import wd as browser

# browser = wd


def name_url():
    # from kora.selenium import wd as browser

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--no-sandbox')
    # browser = webdriver.Chrome('chromedriver', options=chrome_options)

    page_number = 1
    name_url = {}

    while (1):
        try:
            # url for each point of interest page
            url = 'https://www.historic66.com/points-of-interest/?filter=active&state=oklahoma&list=' + str(page_number)
            browser = webdriver.Chrome(options=chrome_options)
            # browser.implicitly_wait(3)
            browser.get(url)
            print("page number: {}".format(page_number))
            for i in range(1, 10):
                select_link = 'body > main > section.content-section > div > div.tabs-block.filter-block > div > div.tabs-panes.content-block-main.border-content > div.left-content.content-ajax > div > div.gallery-block-filter.flex-blocks > a:nth-child(' + str(i) + ')'
                selector = 'body > main > section.content-section > div > div.tabs-block.filter-block > div > div.tabs-panes.content-block-main.border-content > div.left-content.content-ajax > div > div.gallery-block-filter.flex-blocks > a:nth-child(' + str(i) + ') > span'
                selected_name = browser.find_element_by_css_selector(selector).text
                selected_link = browser.find_element_by_css_selector(select_link).get_attribute('href')
                print("selected_name: {} selected_link: {}".format(selected_name, selected_link))

                name_url.update({selected_name: selected_link})

            page_number += 1
        except Exception as e:
            print("Error {}".format(e))
            break
        finally:
            browser.quit()
    return name_url


def name_address(name_url):
    # from kora.selenium import wd as browser

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--no-sandbox')

    name_address = {}

    for name, url in name_url.items():
        try:
            browser = webdriver.Chrome(options=chrome_options)
            browser.implicitly_wait(1)
            browser.get(url)
            try:
                address = browser.find_element_by_css_selector('body > main > section.content-section > div > div.poi-detail.border-content.content-block-main > div.left-content > div.padding-carousel.border-block > div.information-poi > div > div:nth-child(2) > p').text
            except:
                print('{} address NULL'.format(name))
                address = 'NULL'
            try:
                description = browser.find_element_by_css_selector('body > main > section.content-section > div > div.poi-detail.border-content.content-block-main > div.left-content > div.padding-carousel.border-block > div.information-poi > p').text
            except:
                print('{} description NULL'.format(name))
                descrption = 'NULL'
            try:
                image = browser.find_element_by_css_selector('body > main > section.content-section > div > div.poi-detail.border-content.content-block-main > div.left-content > div.padding-carousel.border-block > div.carousel-block > img').get_attribute("src")
            except:
                print('{} image NULL'.format(name))
                image = 'NULL'
            
            print("NAME: {} \n\tADDRESS: {} \n\tDESCRIPTION: {} \n\tIMAGE: {}".format(name, address, description, image))
            name_address.update({name: {'address': address.replace('\n', ' '), 'description': description, 'image': image}})
        except Exception as e:
            print("Error {}".format(e))
            break
        finally:
            browser.quit()

    return name_address

In [ ]:
nurls = name_url()

In [ ]:
jayson = name_address(nurls)

In [ ]:
print(jayson)
print(len(jayson))

In [ ]:
import json

filename = 'historic66.json'
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(jayson, f)